In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.linalg import pinv2
np.set_printoptions(suppress=True,precision=2)
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive',force_remount=True)
root_dir='/content/gdrive/My Drive/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
df=pd.read_csv(root_dir+'AAPL.csv')

In [ ]:
df.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2006,6.999730,7.228929,6.987158,7.228929,201808600
1,1/4/2006,7.265678,7.347880,7.204753,7.250205,154900900
2,1/5/2006,7.236664,7.243433,7.132219,7.193145,112355600
3,1/6/2006,7.277282,7.417508,7.209586,7.378825,176114400
4,1/9/2006,7.420411,7.465863,7.324670,7.354649,168760200


In [ ]:
df.describe()

,Open,High,Low,Close,Volume
count,3105.000000,3105.000000,3105.000000,3105.000000,3.105000e+03
mean,58.800820,59.327872,58.240406,58.796304,1.284830e+08
std,48.717052,49.081938,48.366878,48.731975,1.014688e+08
min,5.002709,5.114889,4.850876,4.900198,1.147590e+07
25%,16.063208,16.329156,15.870760,16.106731,5.019000e+07
50%,45.501145,45.955074,45.026740,45.418461,1.034936e+08
75%,98.564135,99.210053,97.850743,98.667801,1.770629e+08
max,182.589996,183.500000,180.210007,181.720001,8.432424e+08


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105 entries, 0 to 3104
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3105 non-null   object 
 1   Open    3105 non-null   float64
 2   High    3105 non-null   float64
 3   Low     3105 non-null   float64
 4   Close   3105 non-null   float64
 5   Volume  3105 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 145.7+ KB


In [ ]:
 #Let's see a historical view of the closing price

fig=px.line(df,x='Date',y='Close',
           )
fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

fig.show()

In [ ]:
#Check for NA values
df.isna().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

GRAY CORRELATION ANALYSIS

In [ ]:
#Implementing Gray Correlation Degree (GCD):
def GCD(feature_1,feature_2,coef):
  absdiff=abs(feature_2-feature_1)
  max_val=max(absdiff)
  min_val=min(absdiff)
  sum=0
  for i in range(0,len(feature_1)):
    sum+=(min_val+coef*max_val)/(absdiff[i] + coef*max_val)
  correlation=sum/len(feature_1)
  return correlation

MODELING USING EXTREME LEARNING MACHINE

In [ ]:
#Create a new dataframe with only the column except date
data_X= df.filter(['Open','High','Low'])
data_y=df.filter(['Close'])
#Get the number of rows to train the model on
training_data_len = int(np.ceil( len(data_X) * .8 ))

training_data_len


2484

In [ ]:
scaler_x = MinMaxScaler(feature_range=(-1,1))
scaler_y_train= MinMaxScaler(feature_range=(-1,1))
scaler_y_test=MinMaxScaler(feature_range=(-1,1))

In [ ]:
# Create the training data set

X_train= data_X[0:int(training_data_len)]
X_test=data_X[int(training_data_len):]
y_train=data_y[0:int(training_data_len)]
y_test=data_y[int(training_data_len):]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(2484, 3)
(621, 3)
(2484, 1)
(621, 1)


In [ ]:
X_train=X_train.to_numpy()
X_test=X_test.to_numpy()
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [ ]:
#Scaling features 
X_train=scaler_x.fit_transform(X_train)
X_test=scaler_x.fit_transform(X_test)
y_train=scaler_y_train.fit_transform(y_train)
y_test=scaler_y_test.fit_transform(y_test)

In [ ]:
feature_1=X_train[:,1]
feature_2=y_train[:,0]
GCD(feature_1,feature_2,0.5)

0.901831006314738

In [ ]:
#The size of the input layer refers to the number of input features of the dataset.
input_size = X_train.shape[1]
#initialize the number of hidden neurons to 1000.
hidden_size = 100
#we need to initialize our input weights and biases randomly, drawn from a Gaussian distribution.
np.random.seed(0) #Setting seed for reproducibility
input_weights = np.random.normal(size=[input_size,hidden_size])
biases = np.random.normal(size=[hidden_size])
#We will use a Rectified Linear Unit (ReLU) as our hidden layer activation function.
def relu(x):
  return np.maximum(x, 0, x)
#Computing H vector
def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + biases
    H = relu(G)
    return H
#Computing beta
output_weights = np.dot(pinv2(hidden_nodes(X_train)), y_train)
#For testing the model
def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out
prediction=predict(X_test)

In [ ]:
mean_squared_error(y_test,prediction)

0.5780832750845871

In [ ]:
#DeNormalize
y_test=scaler_y_test.inverse_transform(y_test)
prediction=scaler_y_test.inverse_transform(prediction)

In [ ]:
#Creating Dataframe for Trading Strategy
df_1=pd.DataFrame(data=y_test)

In [ ]:
df_1.rename(columns={0:'Actual'},inplace=True)

In [ ]:
df_1['Predicted']=pd.DataFrame(data=prediction)

OSCILLATION BOX THEORY

In [ ]:
 #Calculating Upper Bound
 def upper_bound(T,n):
  upper_bound=[np.nan]*len(T)
  for i in range(len(T)-n):
    upper_bound[i]=max(T[i+1:i+n+1])
  return upper_bound
  
 #Calculating Lower Bound   
 def lower_bound(T,n):
  lower_bound=[np.nan]*len(T)
  for i in range(len(T)-n):
    lower_bound[i]=min(T[i+1:i+n+1])
  return lower_bound
    

In [ ]:
df_1['UB']=upper_bound(df_1['Predicted'],30)

In [ ]:
df_1['LB']=lower_bound(df_1['Predicted'],30)

In [ ]:
df_1.fillna(value=0,inplace=True)

TRADING STRATEGY

In [ ]:
def indicator(data,k):
  if data['Predicted'].iloc[k]>data['UB'].iloc[k]:
    return "buy"
  else:
    return "sell"


In [ ]:
def trading_strategy(data,funds,n):
  volume=0
  actual_funds=[np.nan]*len(data)
  actual_funds[0]=funds
  new_funds=[np.nan]*len(data)
  for i in range(len(data)-n):
    if (indicator(data,i)=="buy"):
      volume+=1
      new_funds[i]=actual_funds[i]-data['Predicted'].iloc[i]
    else:
      volume-=1
      new_funds[i]=actual_funds[i]+data['Predicted'].iloc[i]
    actual_funds[i+1]=new_funds[i]
  return actual_funds,new_funds,volume

In [ ]:
df_1['actual_funds'],df_1['new_funds'],volume=trading_strategy(df_1,10000,30)

In [ ]:
df_1.fillna(value=0,inplace=True)

In [ ]:
df_1['Rate_of_Profit']=((df_1['new_funds']-df_1['actual_funds'])/df_1['actual_funds'])*100

In [ ]:
df_1[0:591]

,Actual,Predicted,UB,LB,actual_funds,new_funds,Rate_of_Profit
0,107.716499,108.138673,114.244352,101.208686,10000.000000,10108.138673,1.081387
1,109.480782,108.561454,114.244352,101.208686,10108.138673,10216.700126,1.074000
2,109.010956,108.445879,114.244352,100.592731,10216.700126,10325.146005,1.061457
3,112.462791,111.633131,114.244352,99.451375,10325.146005,10436.779136,1.081177
4,113.891464,113.431746,114.244352,99.382327,10436.779136,10550.210882,1.086846
...,...,...,...,...,...,...,...
586,178.649994,179.434012,178.307914,161.354294,76080.969853,75901.535840,-0.235846
587,178.020004,178.041154,178.307914,161.354294,75901.535840,76079.576994,0.234569
588,175.300003,174.279137,178.307914,161.354294,76079.576994,76253.856131,0.229075
589,175.240005,176.149054,178.307914,161.354294,76253.856131,76430.005185,0.231003


In [ ]:
print("Average rate of profit:",np.mean(df_1['Rate_of_Profit']))

Average rate of profit: 0.17613523113742982


In [ ]:
fig=px.line(df_1,x=df_1.index,y='Actual',
            labels={'x':'Time'})
fig.update_layout(
    title="Actual vs Predicted Series"
)
fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.add_scatter(x=df_1.index, y=df_1['Predicted'], mode='lines',name='Predicted')
fig.show()